# Inferencia Probabilística

<span style="color: gray">dic 2019</span><br>
[*Alberto Ruiz*](http://dis.um.es/profesores/alberto)


## 1. Cálculo de probabilidades

### Reglas básicas

El cálculo de probabilidades se reduce a tres operaciones elementales:

1) **conjunción** de experimentos:

$$p(x,y) = p(x|y) \, p(y)$$

2) **marginalización** de las variables deseadas:

$$p(x) = \sum_y \, p(x,y)$$

3) **condicionamiento** a los sucesos que nos interesan:

$$p(x|y) = \frac{p(x,y)}{p(y)} $$

La distribución conjunta es una especie de producto cartesiano de posibilidades. La marginalización es simplemente la aplicación de una cierta función (no necesariamente la selección de componentes) a los casos posibles, acumulando los resultados iguales. Y el condicionamiento se reduce a filtrar los casos que cumplen la condición y normalizar.

### Computación probabilística

La implementación de estas operaciones básicas sobre variables aleatorias discretas es sencilla. En este documento vamos a usar un módulo experimental que trata de proporcionar una sintaxis intuitiva.

- Una variable aleatoria se representa mediante un objeto `P` que contiene una lista de sucesos posibles y sus probabilidades asociadas.


- La función `joint` implementa la conjunción de una lista de variables. Admite tanto variables independientes como variables condicionadas expresadas como funciones o diccionarios. Para dos variables usamos el operador `&`. El resultado es una tupla en la que se concatenan los elementos de las variables iniciales.


- El método `conditional` acepta un predicado. Puede abreviarse con el operador `|`


- El método `marginal` acepta cualquier función. Puede abreviarse con `>>` o `<<`.


- Otras utilidades son: `sample`, `mean`, `median`, `mode`, `evidence`, `transform`, `prob`, `show`, `showhdi`.


- Importamos `repeat` y `Counter` y definimos `uniform`, `bernoulli`, `S`, `equal`.

Veamos algunos ejemplos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import umucv.prob as pr

from itertools import repeat
from collections import Counter

def uniform(s):
    return pr.P({x : 1 for x in s})

def bernoulli(p,a,b):
    return pr.P({a:p, b:1-p},norm=False)

def S(k):
    return lambda x: x[k]

def equal(k,v):
    return lambda x: x[k] == v

### Variables aleatorias

Una variable aleatoria se caracteriza por las probabilidades que tienen los sucesos posibles. Un caso muy frecuente es la distribución uniforme:

In [ ]:
def uniform(s):
    return pr.P({x : 1 for x in s})

Una moneda equilibrada se puede modelar así:

In [ ]:
moneda = uniform(["cara", "cruz"])

In [ ]:
moneda

In [ ]:
print(moneda)

In [ ]:
#plt.rcParams['figure.figsize'] = (10, 5)

In [ ]:
pr.show(moneda)

Una operación natural sobre una variable aleatoria es simular un cierto número de "realizaciones" del experimento.

In [ ]:
lanzamientos = moneda.sample(50)
print(lanzamientos)
Counter(lanzamientos)

Una variable aleatoria de tipo Bernoulli modela un suceso "binario" que tiene una cierta probabilidad de éxito.

In [ ]:
# experimento que produce a con probabilidad p
def bernoulli(p,a,b):
    return pr.P({a:p, b:1-p},norm=False)

In [ ]:
trucada = bernoulli(0.6,"cara","cruz")
print(trucada)
pr.show(trucada)

In [ ]:
lanzamientos = trucada.sample(50)
print(lanzamientos)
Counter(lanzamientos)

Un dado genérico de n caras, equilibrado:

In [ ]:
gdado = lambda n: uniform(range(1,n+1))

dado = gdado(6)
print(dado)
pr.show(dado)

Un dado desequilibrado:

In [ ]:
dadomalo = pr.P({1:1, 2:1, 3:1, 4:1, 5:1, 6:2})
print(dadomalo)
pr.show(dadomalo)

In [ ]:
dadomalo.sample(10)

Jaynes propone medir una probabilidad (mejor dicho, el *odds ratio*) en *decibelios*.

In [ ]:
pr.evidence(pr.P({'A':1000, 'B':1})['A'])

### Conjunción de experimentos independientes

Lanzamiento de un dado y una moneda:

In [ ]:
experimento = dado & moneda

print(experimento)

print(experimento.sample(5))

Lanzamiento de 3 monedas:

In [ ]:
print(pr.joint(repeat(moneda,3)))

### Marginalización

La suma de dos dados:

In [ ]:
print( sum << (dado & dado) )

El número de caras al lanzar cuatro monedas:

In [ ]:
print( pr.joint(repeat(moneda,4)) >> (lambda x: Counter(x)['cara']) )

La probabilidad de obtener al menos 3 caras al lanzar 4 monedas. Expresado como la marginalización de un predicado:

In [ ]:
print( pr.joint(repeat(moneda,4)) >> (lambda x: Counter(x)['cara'] >= 3) )

Lo mismo expresado con el método `prob`:

In [ ]:
pr.joint(repeat(moneda,4)).prob( lambda x: Counter(x)['cara'] >= 3 ) 

Para abreviar el código añadimos una función para extraer un componente dado de una tupla.

In [ ]:
def S(k):
    return lambda x: x[k]

Al marginalizar una variable independiente de otras se obtiene la misma variable:

In [ ]:
print( (dado & trucada & moneda) >> S(1) )

La suma de $n$ dados:

In [ ]:
dados = lambda n: pr.joint(repeat(dado,n)) >> sum

El 95% de las veces la suma de 3 dados está entre 6 y 15:

In [ ]:
pr.show(dados(3))
pr.showhdi(dados(3),95)

El máximo de 2 dados:

In [ ]:
pr.show( max << (dado & dado) )

La diferencia entre la máxima y la mínima puntuación al lanzar 3 dados:

In [ ]:
exper = (lambda x: max(x) - min(x)) << pr.joint(repeat(dado,3)) 

pr.show(exper)

Características de la distribución:

In [ ]:
exper.mean()

In [ ]:
exper.median()

In [ ]:
exper.mode()

### Teorema central del límite

Bajo condiciones bastante generales la suma de variables aleatorias arbitrarias se aproxima a una campana de Gauss.

In [ ]:
raro = pr.P({1: 11, 2: 5, 3:1 , 4: 3, 5: 7, 6:9})
pr.show(raro)

In [ ]:
pr.show (sum << pr.joint(repeat(raro,5)))

### Monty Hall

Hay tres puertas. Detras de una hay un tesoro. Elegimos una de ellas. Nos abren otra, que está vacía. ¿Interesa cambiar?

In [ ]:
doors = {"A","B","C"}
prize = uniform(doors)
guess = uniform(doors)

premio, adiv, abierta, cambiar = 0,1,2,3

def open(s):
    return uniform( doors - {s[premio] , s[adiv]} )

def change(s):
    return uniform( doors - {s[adiv] , s[abierta]} )

monty = prize & guess & open & change
print(monty)

¿Cuál es la probabilidad de que la puerta con el premio sea la misma que se adivinó al principio?

In [ ]:
monty.prob( lambda s: s[premio]==s[adiv])

¿Cuál es la probabilidad de que la puerta con el premio sea la que dejan cerrada?

In [ ]:
monty.prob( lambda s: s[premio]==s[cambiar])

### Newton - Pepys

Probabilidad de obtener al menos $n$ seises al lanzar $6n$ dados ([Newton-Pepys problem](https://en.wikipedia.org/wiki/Newton%E2%80%93Pepys_problem))

In [ ]:
def newtonpepys(n):
    exper = pr.joint(repeat(dado,6*n))
    def ok(r):
        return Counter(r)[6] >= n
    return exper.prob(ok)

In [ ]:
%%time

newtonpepys(1)

In [ ]:
6**6

Este problema es intratable para $n>1$. Pero se puede resolver de forma aproximada muestreando.

In [ ]:
def newtonpepyssampled(n, size=10000):
    return np.mean([Counter(dado.sample(6*n))[6] >= n for j in range(size)])

In [ ]:
newtonpepyssampled(1)

In [ ]:
newtonpepyssampled(2)

In [ ]:
newtonpepyssampled(3)

### Probabilidad condicionada

¿Qué probabilidad tienen las puntuaciones de un dado si sale un resultado mayor que 3?

In [ ]:
print( dado | (lambda x: x>3) )

¿Qué probabilidades tienen las posibles puntuaciones al lanzar dos dados si suman 5?

In [ ]:
print( dado & dado | (lambda x: sum(x)== 5) )

### Experimentos dependientes

Lanzamos una moneda. Si sale cara tiramos un solo dado. Si sale cruz tiramos dos.

In [ ]:
experimento = moneda & {'cara': dados(1), 'cruz': dados(2)}

print( experimento )

¿Qué distribución de probabilidad tiene la puntuación total obtenida?

In [ ]:
pr.show( experimento >> S(1) )

El mismo resultado con el método `transform`, que automáticamente construye la conjunta y marginaliza:

In [ ]:
pr.show( moneda.transform( {'cara': dados(1), 'cruz': dados(2)} ), ticks=True)

Lanzamos un primer dado. La puntuación obtenida nos dice el número de dados que debemos lanzar para obtener la puntuación final. ¿Cuál es la distribución de puntuaciones de este experimento?

In [ ]:
pr.show((dado & dados) >> S(1))

In [ ]:
pr.show( dado.transform(dados) )

## 2. Inferencia Bayesiana

### Regla de Bayes

En muchos casos el experimento se define mediante una probabilidad "a priori" $p(A)$ y un "modelo de observación" $p(B|A)$, pero lo que nos interesa es la probabilidad condicionada contraria $p(A|B)$.

Por ejemplo, en el experimento anterior, donde lanzamos uno o dos dados dependiendo de un lanzamiento de moneda que no hemos visto, si la puntuación total obtenida es 6, ¿que es más probable que haya salido en la moneda?

In [ ]:
experimento = moneda & {'cara': dados(1), 'cruz': dados(2)}

De nuevo, para simplificar el código, definimos una función auxiliar para comprobar si un elemento de una tupla tiene un valor concreto.

In [ ]:
def equal(k,v):
    return lambda x: x[k] == v

In [ ]:
print(( experimento | equal(1 , 6)) )

### Clasificación bayesiana

Deseamos reconocer dos clases de objetos a partir de la observación de una cierta propiedad. En la clase C1 esta propiedad toma valores más bajos y concentrados, y en la clase C2 suele tomar valores más altos y dispersos. Además sabemos que la clase C2 es algo más probable. Esta situación puede modelarse de la forma siguiente (el uso de dados aquí es una forma rápida de generar distribuciones con la forma deseada).

In [ ]:
prior  = bernoulli(0.4, 'C1', 'C2')
sensor = {'C1': dados(3), 'C2': dados(5)}

pr.show(sensor['C1'],alpha=0.5)
pr.show(sensor['C2'],alpha=0.5)

Las realizaciones más frecuentes son las siguientes:

In [ ]:
def typical(p):
    return sorted(p.items(), key=lambda x: x[1], reverse=True)[:10]

model = prior & sensor
typical( model )

Pero, por supuesto, la clase no es observable. Sin embargo, podemos actualizar su distribución de probabilidad si observamos la propiedad:

In [ ]:
posterior = model | equal(1, 16)

print (posterior )

Es importante calcular el error de clasificación intrínseco del problema (Error de Bayes). Es la probabilidad de error de la regla de decisión óptima (decidir la clase más probable a posteriori), que se calcula como la integral del mínimo de las densidades condicionadas ponderadas con las probabilidades a priori. Es simplemente el área de solapamiento.

In [ ]:
pr.show(sensor['C1'],alpha=0.5)
pr.show(sensor['C2'],alpha=0.5)
x,y = zip(*sensor['C1'].items())
plt.plot(x,np.array(y)*prior['C1']);
x,y = zip(*sensor['C2'].items())
plt.plot(x,np.array(y)*prior['C2']);

In [ ]:
bayeserror = sum([ min( prior['C1']*sensor['C1'][x] , prior['C2']*sensor['C2'][x] ) for x in range(31)] )
bayeserror

### Test de enfermedad

Es el caso más simple de decisión bayesiana. De nuevo hay que distinguir dos clases, pero en este caso la propiedad observada es binaria, y por tanto se caracteriza por dos probabilidades: la "sensibilidad" y la "selectividad".

El ejemplo siguiente es un caso hipotético, con números inventados para ilustrar los conceptos. No tiene nada que ver con ningún análisis médico real.

Supongamos que una cierta enfermedad aparece en una de cada 1000 personas.

In [ ]:
gente = bernoulli(1/1000,'enf','sano')
gente

Consideremos una prueba con una sensibilidad del 99% y una selectividad del 95%. Esto significa que detecta correctamente al 99% de los enfermos y al 95% de los sanos. (Estas probabilidades son diferentes porque un falso negativo es más grave que un falso positivo. Normalmente no pueden ser ambas muy altas y se toma un compromiso razonable).

In [ ]:
test = {'enf':  bernoulli(99/100,'+','-'),
        'sano': bernoulli(95/100,'-','+')}    
    
caso1 = gente & test

print(caso1)

Aunque una prueba dé positivo, el estado "sano" sigue siendo más probable:

In [ ]:
S(0) << (caso1 | equal(1, '+' ))
print(_)

Esto es debido a que hay muchos más sanos que enfermos. El 5% de falsos positivos del conjunto "sano" es mucho mayor que el 99% del conjunto "enf". 

Dos formas de expresar lo mismo:

In [ ]:
(caso1 |  equal(1, '+' )).prob(equal(0,'sano'))

In [ ]:
((caso1 |  equal(1, '+' )) >> S(0) )['sano']

Hagamos un segundo test:

In [ ]:
caso2 = caso1 & (lambda s: test[s[0]])
caso2

In [ ]:
S(0) << (caso2 | equal(1,'+') )

In [ ]:
S(0) << (caso2 | (lambda x: x[1:] == ('+','+')) )

Si también da positivo sigue siendo más probable el estado "sano", aunque la situación está dudosa.

In [ ]:
caso3 = caso2 & (lambda s: test[s[0]])
print(caso3)

Un tercer positivo ya sí hace muy probable el estado "enf".

In [ ]:
caso3 | (lambda x: Counter(x[1:])['+'] == 3)

Es interesante observar que un test positivo y otro negativo no se "contrarrestan":

In [ ]:
S(0) << (caso2 |  (lambda x: x[1:] == ('-','+'))) 

In [ ]:
S(0) << (caso2 |  (lambda x: x[1:] == ('+','-'))) 

Una forma de visualizar la situación anterior es expresar la densidad conjunta de las dos formas alternativas:

$$ P(A,B) = P(A|B) P(B) = P(B|A) P(A) $$

La regla de Bayes permite pasar de una a otra.

![bayes](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/graph/bayes22.png)

### Observaciones binarias condicionalmente independientes

En este caso cada observación (test) suma una contribución a la *log likelihood ratio*, positiva o negativa según el resultado, que depende de la calidad del test, que queda especificada por $s\equiv P\{+|T\}$ y $e\equiv P\{-|F\}$. 

Contrariamente a la intuición, si un test muy sensible ($s$ cercano a 1) produce un resultado positivo, puede no aportar casi nada de información. Todo depende de la especificidad. Lo que sí aportaría muchísima información sería un resultado negativo. Recíprocamente, si un test muy selectivo ($e$ cercano a 1, con muy baja probabilidad de falso positivo) produce un resultado negativo, tampoco aporta información. Todo depende de la sensibilidad. Lo que aportaría información es un resultado positivo.

Dicho de otro modo, un test poco sensible, si se dispara es buena señal, y uno muy sensible, puede ser que se dispare por cualquier causa si es poco selectivo.


[deciban]: https://en.wikipedia.org/wiki/Hartley_(unit)

Por tanto, lo interesante quizá sería caracterizar un test con el "delta" de "logodds" que aporta cada resultado. Usaremos [decibans][deciban]. Por ejemplo:

     s      e      +       -
    99%    95%    +13     -20
    99%    99%    +20     -20
    75%    75%    +5      -5
    95%    70%    +5      -12
    99%    50%    +3      -17

(Una $e$ menor de 0.5 hace que un resultado + sea frecuente y el - sea infrecuente en los dos estados T,F, por lo que tiene poca utilidad. Y si ambos $s$ y $e$ son menores de 0.5 simplemente habría que cambiar la etiqueta del resultado.)

Por otro lado un test poco sensible, si es selectivo, aporta información siempre que su sensibilidad $s > 1-e$, porque si no llega un momento en que funciona de forma inversa.

      s      e      +       -
     10%    99%    +10     -0.4
      2%    99%    +3      -0.04
    0.5%    99%    -3      +0.02
    0.1%    99%    -10     +0.04

In [ ]:
def sig(z):
    return 1/(1+10**(-z/10))


def lb(p):
    return 10*np.log10(p)

def llr(p):
    return lb(p) - lb(1-p)

In [ ]:
z = np.linspace(0,30,100);
plt.plot(z,sig(z)); plt.grid();
plt.title('logistic (sigmoidal) function')
plt.xlabel('log prob ratio (deciban)'); plt.ylabel('probabilidad');

print('db      p      1-p')
for z in range(5,51,5):
    print('{:2}  {:.5f}  {:5.2f}%'.format(z,sig(z),100*(1-sig(z))))

In [ ]:
def mkTest(s,e):
    p = lb(s)   - lb(1-e)
    n = lb(1-s) - lb(e) 
    return lambda x: p if x else n

In [ ]:
T1 = mkTest(99/100, 95/100)

In [ ]:
sig( llr(1/1000) + T1(True) + T1(True) + T1(True) )*100

In [ ]:
sig( llr(1/1000) )*100

In [ ]:
sig( llr(1/1000) + T1(True) + T1(False) )*100

In [ ]:
T1(True), T1(False)

In [ ]:
T = mkTest(999/1000, 80/100)
T(True), T(False)

In [ ]:
T = mkTest(99.9/100, 50/100)
T(True), T(False)

In [ ]:
sig( T(True))

In [ ]:
T = mkTest(80/100, 50/100)
T(True), T(False)

In [ ]:
T = mkTest(99/100, 50/100)
T(True), T(False)

In [ ]:
T = mkTest(90/100, 99/100)
T(True), T(False)

In [ ]:
T = mkTest(40/100, 99/100)
T(True), T(False)

In [ ]:
T = mkTest(2/100, 99/100)
T(True), T(False)

In [ ]:
T = mkTest(0.5/100, 99/100)
T(True), T(False)

In [ ]:
T = mkTest(0.1/100, 99/100)
T(True), T(False)

### Dados platónicos

Tenemos una colección de 5 dados (para juegos de rol o algo así) con 4, 6, 8, 12 y 20 caras. Lanzamos uno de ellos elegido al azar, y sin mirar cuál es observamos la puntuación. ¿Qué dado es más probable?

In [ ]:
exper = uniform([4,6,8,12,20]) & gdado

In [ ]:
plt.figure(figsize=(18,4))
pr.show(exper,rotation=90)

In [ ]:
pr.show(exper >> S(1),ticks=True)

In [ ]:
pr.show( exper | equal(1, 11) )

In [ ]:
pr.show( exper | equal(1, 6) )

Vamos a hacer lo mismo, elegir un dado al azar pero en este caso observamos la suma de tres lanzamientos.

In [ ]:
exper = uniform([4,6,8,12,20]) & (lambda x: pr.joint(repeat(gdado(x),3)) >> sum)

plt.figure(figsize=(22,4))
pr.show(exper,rotation=90)

In [ ]:
plt.figure(figsize=(16,4))
pr.show(exper >> S(1))

In [ ]:
plt.figure(figsize=(22,4))
plt.subplot(1,5,1); pr.show(exper | equal(1, 5 ))
plt.subplot(1,5,2); pr.show(exper | equal(1, 8 ))
plt.subplot(1,5,3); pr.show(exper | equal(1, 9 ))
plt.subplot(1,5,4); pr.show(exper | equal(1, 13))
plt.subplot(1,5,5); pr.show(exper | equal(1, 25 ))

Si observamos el mínimo de tres lanzamientos:

In [ ]:
exper = uniform([4,6,8,12,20]) & (lambda x: pr.joint(repeat(gdado(x),3)) >> min)

plt.figure(figsize=(22,4))
pr.show(exper,rotation=90)

In [ ]:
plt.figure(figsize=(16,4))
pr.show(exper >> S(1), ticks=True)

In [ ]:
plt.figure(figsize=(22,4))
plt.subplot(1,5,1); pr.show(exper | equal(1, 1 ))
plt.subplot(1,5,2); pr.show(exper | equal(1, 2 ))
plt.subplot(1,5,3); pr.show(exper | equal(1, 3 ))
plt.subplot(1,5,4); pr.show(exper | equal(1, 4))
plt.subplot(1,5,5); pr.show(exper | equal(1, 10 ))

Y el máximo:

In [ ]:
exper = uniform([4,6,8,12,20]) & (lambda x: pr.joint(repeat(gdado(x),3)) >> max)

plt.figure(figsize=(22,4))
pr.show(exper,rotation=90)

In [ ]:
plt.figure(figsize=(16,4))
pr.show(exper >> S(1), ticks=True)

In [ ]:
plt.figure(figsize=(22,4))
plt.subplot(1,5,1); pr.show(exper | equal(1, 5 ))
plt.subplot(1,5,2); pr.show(exper | equal(1, 4 ))
plt.subplot(1,5,3); pr.show(exper | equal(1, 6 ))
plt.subplot(1,5,4); pr.show(exper | equal(1, 8))
plt.subplot(1,5,5); pr.show(exper | equal(1, 12 ))

### A girl named Florida

Se trata de un problema de inferencia muy contraintuitivo que apareció en el [blog de Allen Downey](http://allendowney.blogspot.com.es/2011/11/girl-named-florida-solutions.html).

Consideremos familias con dos hijos/as, a los que ponemos nombre, más o menos de cualquier manera. Las realizaciones del experimento tienen cuatro atributos: el sexo y el nombre del primero hijo y el sexo y nombre del segundo.

In [ ]:
gente = bernoulli(1/2, "boy", "girl")

name = {"boy" : pr.P({"Pepe":6, "Juan":4}),
        "girl": pr.P({"Ana":4, "Eva":4 , "María":2})}

child = name & gente
family = child & child
print(family)

Podemos marginalizar los nombres:

In [ ]:
names = lambda x: (x[0], x[2])
print(names << family)

O los sexos, obteniéndose las proporciones esperadas:

In [ ]:
sexes = lambda x: (x[1], x[3])
sexes << family

Primera pregunta: si **el primer** hijo es niña, ¿qué probabilidad hay de que la familia tenga dos hijas?

In [ ]:
print( sexes << (family  | equal(1,'girl')) )

Expresado de otra manera:

In [ ]:
twogirls = lambda x: Counter(sexes(x))['girl'] == 2

print( twogirls << (family  | equal(1, 'girl')) )

In [ ]:
(family  | equal(1, 'girl')).prob(twogirls)

Segunda pregunta: si **algún** hijo es niña, ¿qué probabilidad hay de tener dos hijas?

In [ ]:
atleastonegirl =lambda x: Counter(sexes(x))['girl'] >= 1

sexes << (family  | atleastonegirl)

In [ ]:
print( names << (family | atleastonegirl) )

In [ ]:
(family  | atleastonegirl ).prob( twogirls )

La probabilidad es menor que antes, ya que hay un caso posible más. La condición "primer hijo = niña" abarca menos casos (2) que "algún hijo niña" (3). (Podría pensarse que la condición de que haya alguna niña en cualquier orden (en vez del primero) favorece que haya dos en total, pero esto no es así.)

Pero la prengunta realmente interesante es la siguiente:
    
¿Cuál es la probabilidad de que haya dos niñas si alguna es una niña que se llama "María"?

Este nombre en realidad no tiene nada de especial, se trata de añadir una propiedad adicional a la condición. Lo interesante es que esa propiedad adicional no puede influir en la distribución de sexos. El nombre que pongamos a un hijo no puede afectar al sexo del otro.

Pero veamos el resultado:

In [ ]:
atleastonegirlcalledMaría = lambda s: s[:2]==("María","girl") or s[2:]==("María","girl")

print( family | atleastonegirlcalledMaría )

In [ ]:
sexes << ( family | atleastonegirlcalledMaría )

In [ ]:
( family | atleastonegirlcalledMaría ).prob(twogirls)

El resultado se aproxima a 0.5 correspondiente a la preguna inicial, donde sabemos que el primer hijo es niña. Y además, se aproximará más cuanto más improbable sea el nombre de la niña (la propiedad adicional). ¿Qué está pasando? 

Queda claro con el diagrama del [blog de Allen Downey](http://allendowney.blogspot.com.es/2011/11/girl-named-florida-solutions.html). Cuando sabemos que los casos posibles se reducen a la unión de las dos franjas Gx, los casos favorables se reducen también a la franja más oscura Gx,G + G,Gx.

![Florida](https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/data/graph/florida.png)

Las probabilidades anteriores no tienen que ver con causas y efectos, solo reflejan las frecuencias relativas de las diferentes combinación de sexo y nombre en subconjuntos de interés.

### Inferencia acerca de una probabilidad

Cuando las variables aleatorias son continuas podemos discretizarlas y seguir utilizando las herramientas anteriores. Son las **técnicas de *grid***, útiles en problemas de dimensión pequeña.

Supongamos un experimento aleatorio que en principio puede tener una probabilidad alrededor de 1/2. ¿Qué diríamos si  observamos 5 éxitos consecutivos?

In [ ]:
from scipy.stats import beta
import numpy as np

La distribución beta es muy útil para expresar información a priori sobre la probabilidad de un suceso.

In [ ]:
p = np.linspace(0,1,50)
b = beta(2,2).pdf(p)

plt.plot(p, b);
plt.grid(ls='dotted')

In [ ]:
prior = pr.P(dict(zip(p,b)))
pr.show(prior)
pr.showhdi(prior,90/100)

In [ ]:
def coin(p):
    return bernoulli(p,'c','+')

def coins(n,p):
    return pr.joint(repeat(coin(p),n))

In [ ]:
coins(4,0.25)

In [ ]:
exper = (lambda p: coins(5,p)) & prior

print(len(exper.items()))

list(exper.items())[:3]

In [ ]:
pr.show(exper >> (lambda x: Counter(x[:5])['c']))

In [ ]:
exper2 = exper >>  (lambda x: (Counter(x[:5])['c'],x[-1] ) )
list(exper2.items())[:8]

In [ ]:
plt.figure(figsize=(7,7)); plt.grid(ls='dotted')
pr.show(prior,alpha=0.5)
post = ( exper2 | (lambda x: x[0] == 4) ) >> S(1)
pr.show( post , alpha=0.5)
pr.showhdi( post, 90/100)

In [ ]:
plt.figure(figsize=(7,7)); plt.grid(ls='dotted')
pr.show(prior,alpha=0.5)
post = ( exper2 | (lambda x: x[0] == 5) ) >> S(1)
pr.show( post , alpha=0.5)
pr.showhdi( post, 90/100)

In [ ]:
post.mean()

In [ ]:
post.median()

In [ ]:
post.mode()

En realidad, la densidad Beta y la Binomial son [conjugadas](https://en.wikipedia.org/wiki/Conjugate_prior), por lo que estos resultados se obtienen de forma analítica:

In [ ]:
plt.figure(figsize=(5,5)); plt.grid(ls='dotted')
plt.plot(p,beta(2,2).pdf(p));
plt.plot(p,beta(2+4,2+5-4).pdf(p));
plt.plot(p,beta(2+5,2+5-5).pdf(p));

In [ ]:
plt.figure(figsize=(5,5)); plt.grid(ls='dotted')
plt.plot(p,beta(1,1).pdf(p));
plt.plot(p,beta(1+1,1+1).pdf(p));
plt.plot(p,beta(1+5,1+5).pdf(p));

### Gaussian update

Algunas distribuciones de probabilidad tienen una forma matemática que permite calcular de forma **analítica** la regla de Bayes. Son las [distribuciones conjugadas](https://en.wikipedia.org/wiki/Conjugate_prior). (Ver también [estas notas](https://ocw.mit.edu/courses/mathematics/18-05-introduction-to-probability-and-statistics-spring-2014/readings/MIT18_05S14_Reading15a.pdf).) Un caso importante es la distribución normal, que es conjugada de ella misma. En el caso 1D, si el prior es

$$\mathcal p(x) \sim N[\mu, \sigma]$$

y la verosimilitud o modelo de observación es 

$$p(s\mid x) \sim \mathcal N[0,r]$$

La distribución *a posteriori* tras observar el valor concreto $s_o$ es:

$$p(x\mid s_o) = \mathcal N[\mu',\sigma']$$

donde

\begin{aligned}
\mu' &= \mu + K(s_o-\mu)\\
\sigma'^2 &= (1-K)\sigma^2
\end{aligned}

donde K es la "ganancia de Kalman" que cuantifica la influencia que tiene la observación, que será mayor cuanto menor ruido tenga el modelo

$$K = \frac{\sigma^2}{\sigma^2+r^2} $$

Podemos expresarlo así: 

- Inicialmente creemos que la realidad es $\mu$, con incertidumbre $\sigma$.

- Tenemos un aparato capaz de medir la magnitud real con un ruido $r$.

- Llega un nuevo dato experimental $s_o$ que produce un error de predicción $\epsilon = s_o-\mu$. (Idealmente $\epsilon$ sería cero si el mecanismo de medida no tuviera ruido y nuestra información fuera perfecta.)

- La discrepancia observada $\epsilon$ nos obliga a actualizar o revisar nuestra creencia: $\mu' = \mu + K\epsilon$.

- Y a la vez la incertidumbre se reduce en un factor $\sqrt{(1-K)}$.

La importancia de la actualización viene dada por el factor $K$ que pondera las dos incertidumbres, la inicial y la del sensor. Si $r \ll \sigma$ entonces $K\simeq1$ y $\mu'\simeq s_o$, otorgamos mucha confianza a esta última observación y la incertidumbre se reduce mucho. Si el sensor es muy ruidoso, $r \gg \sigma$, ocurre lo contrario: $K\simeq 0$ y la observacion apenas modifica $\mu$ y $\sigma$.

Este resultado puede demostrarse fácilmente teniendo en cuenta que ...

En este apartado comprobamos las expresiones anteriores para el caso 1D con las herramientas de computación probabilística de este notebook, discretizando el dominio.

In [ ]:
from scipy.stats import norm
import numpy as np

In [ ]:
x = np.linspace(-7,7,100)

def N(m,s):
    G = norm(m,s).pdf(x)
    return pr.P(dict(zip(x,G)))

In [ ]:
prior = N(1,1.2)
likelihood = lambda x: N(x, 0.5)

pr.show(prior, edgecolor=None)
pr.show(likelihood(-3),alpha=0.6, edgecolor=None)
plt.legend(['prior','likelihood']);
plt.title('$p(x|s=-3)\hspace{3} p(s)$');

In [ ]:
obs = prior.transform(likelihood)
pr.show(prior, edgecolor=None)
pr.show(obs,alpha=0.5, edgecolor=None, color='red')
plt.title('$p(x)=\int p(x|s)p(s)ds$');
plt.legend(['prior','observation']);

In [ ]:
pr.show(prior,edgecolor=None)
pr.show(likelihood(-2.05),edgecolor=None)

post = ((prior & likelihood) | (lambda x: -2.1 <= x[1] < -2)   )>>S(0)

pr.show( post ,alpha=0.5,edgecolor=None, color='green')
plt.legend(['prior','likelihood','posterior']);
plt.title('$p(\mu\sigma\,|\,x=-2)$');

In [ ]:
def std(p):
    m = p.mean()
    return np.sqrt((p >> (lambda x: (x-m)**2)).mean())

In [ ]:
prior.mean(), std(prior)

In [ ]:
post.mean(), std(post)

In [ ]:
sp = std(prior)
mp = prior.mean()
r  = std(likelihood(-2.05))

K = sp**2/(sp**2+r**2)

pm = mp + K*(-2.05 - mp)
ps = np.sqrt((1-K)*sp**2)

pm, ps

Existen expresiones análogas para distribuciones Gaussianas multivariables. Y es en esta situación en la que se saca realmente partido de la inferencia Bayesiana con distribuciones Gaussianas. Debido a la dependencia entre variables, se puede reducir la incertidumbre sobre alguna de ellas dada la observación de otras. Por ejemplo, a partir de un modelo Gaussiano para $p(x,y,z)$, podemos calcular analíticamente distribuciones condicionadas del tipo $p(y \mid x-2z = 1.3)$. Esto es la base del [filtro de Kalman](https://en.wikipedia.org/wiki/Kalman_filter) y de los [procesos Gaussianos](https://en.wikipedia.org/wiki/Gaussian_process). Los detalles de esto se explican en [este notebook](Kalman.ipynb).

### Billar raro

El problema que aparece en el [blog de Jake VanderPlas](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/). La probablidad de éxito de un suceso es desconocida (pero uniforme entre cero y uno). Si vamos perdiendo 5 a 3 en una partida a 6 puntos, ¿cuál es la probabilidad de remontar y ganar 6 a 5?

La ventaja de 5 a 3 da una estimación de máxima verosimilitud $\hat p = 3/8$ = {{3/8}}, y por tanto una probabilidad de tres puntos consecutivos $\hat p^3$ = {{ '{:.3f}'.format((3/8)**3) }}.

El análisis bayesiano da un resultado distinto. Para un prior uniforme sobre $p$ la distribución posterior es:

$$P(p|3-5)=\frac{p^3(1-p)^5}{B(4,6)}$$

In [ ]:
from scipy.special import beta as Beta

post = lambda p: p**3*(1-p)**5/Beta(4,6)
p = np.linspace(0,1,100)
plt.plot(p,post(p));

La probabilidad de tres puntos consecutivos con esta distribución es:

$$P(6-5) = \frac{B(7,6)}{B(4,6)} $$

In [ ]:
Beta(7,6)/Beta(4,6)

Es casi el doble que la estimación más verosimil puntual anterior.

Vamos a reproducir este resultado analítico mediante la técnica de *grid*, que consiste simplemente en discretizar las distribuciones y operar con las mismas herramientas usadas hasta ahora.

In [ ]:
p = np.linspace(0,1,50)
b = beta(1,1).pdf(p)
bola1 = pr.P(dict(zip(p,b)))

In [ ]:
bola2 = lambda b1: pr.joint(repeat(bernoulli(b1,0,1) ,8)).marginal(sum) 
exper = (bola2 & bola1).conditional(lambda s: s[0] == 5).marginal(S(1))
list(exper.items())[:5]

In [ ]:
pr.show(bola1,alpha=0.5)
pr.show(exper,alpha=0.5)
pr.showhdi(exper,99/100)

In [ ]:
seguir = (lambda b1: pr.joint(repeat(bernoulli(b1,0,1) ,3))) & exper
list(seguir.items())[:5]

In [ ]:
seguir.marginal(lambda s: sum(s[:3]))

In [ ]:
seguir.prob(lambda s: sum(s[:3])==0)

In [ ]:
exper.mode()

In [ ]:
exper.mode()**3

De nuevo comprobamos que trabajar con el estimador más probable, o más verosímil, sin tener en cuenta la dispersión no es lo ideal.

La segunda parte de este experimento se ha hecho generando todas las posibilidades de los 3 siguientes lanzamientos. Para mayor eficiencia, en las técnicas de grid este paso se abreviaría calculando directamente la probabilidad a posteriori con la probabilidad analítica de obtener 3 éxitos.

### Shannon

Ejemplo en Shannon (1948).

In [ ]:
symbol = uniform(['A','B','C'])

def channel(x):
    if x == 'A': return pr.P({'a' : 1})
    if x == 'B': return pr.P({'b': 0.8, 'c':0.2})
    if x == 'C': return pr.P({'b': 0.2, 'c':0.8})

In [ ]:
symbol & symbol

In [ ]:
symbol & channel

In [ ]:
system = (lambda x : (''.join(x[0::2]),''.join(x[1::2]))) << pr.joint(repeat((symbol & channel), 5)) 
#system

In [ ]:
S(0) << (system | equal(1,'aabcb'))